# Test segmentation model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys,inspect
#sys.path.insert(0,"..")
os.chdir('..')

In [ ]:
import numpy as np
import cv2
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch

from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset

from dataset import GLAS_dataloader, POLYPS_dataloader, NYUDV2_dataloader
from models.unet import build_unet
from models.LeViTUNet128s import Build_LeViT_UNet_128s
from models.LeViTUNet192 import Build_LeViT_UNet_192
from models.LeViTUNet384 import Build_LeViT_UNet_384
from models.unetplusplus import NestedUNet

torch.manual_seed(0)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

# Device
DEVICE = "cpu" #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

EXPERIMENT_NAME = "nyu_experiments/nyu_nestunet_cb_ts_h/"
ROOT_DIR = os.path.abspath(".")
LOG_PATH = os.path.join(ROOT_DIR, "logs", EXPERIMENT_NAME)

data_folder = 'datasets/NYUDV2'
#model_path = 'logs/glas_experiments/{}/{}.pth'.format(EXPERIMENT_NAME, EXPERIMENT_NAME)
model_path = 'logs/nyu_experiments/nyu_nestunet_cb_ts_h/nyu_nestunet_cb_ts_h.pth'
model_path

In [ ]:
train_dataset = NYUDV2_dataloader("datasets/NYUDV2")
test_dataset = NYUDV2_dataloader("datasets/NYUDV2", is_train=False)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)

In [ ]:
#model = Build_LeViT_UNet_384(num_classes=1, pretrained=True)
model = NestedUNet(num_classes=40)

checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint)
model = model.to(DEVICE)
model.eval()

In [ ]:
def visualize(idx, **images):
    """Plot images in one row.""" 
    n = len(images)
    fig = plt.figure(figsize=(20, 10))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).lower(), fontsize=20)
        plt.imshow(image, cmap='gray')
        plt.axis("off")
        #plt.tight_layout()
    #plt.savefig("../outputs/vis/compare-segs/{}.png".format(idx), facecolor="white", bbox_inches = 'tight')
    plt.show()

In [ ]:
len(test_dataloader)
# y[y==1] = 255

In [ ]:
LOG_PATH

In [ ]:
if not os.path.exists(os.path.join(LOG_PATH, "vis_masked")):
    os.mkdir(os.path.join(LOG_PATH, "vis_masked"))
    os.mkdir(os.path.join(LOG_PATH, "vis_masked", "imgs"))
    os.mkdir(os.path.join(LOG_PATH, "vis_masked", "gts"))
    os.mkdir(os.path.join(LOG_PATH, "vis_masked", "preds"))

In [ ]:
# Load color map
cmap = np.load('datasets/NYUDV2/cmap.npy')

for batch_idx, data in enumerate(test_dataloader):
    img, target = data["partial_image1"].to(DEVICE), data["mask"].to(DEVICE) # partial_image1 instead of image
    output = torch.sigmoid(model(img.float()))
    
    img = (img[0].permute(1,2,0).detach().cpu().numpy()+1)/2
    img = (img*255).astype(np.uint8)
    #img= cv2.cvtColor(img,cv2.COLOR_RGB2BGR) # uncomment when saving images
    gt = target.squeeze().data.cpu().numpy()
    gt = cmap[gt]
    
    output = torch.softmax(output, dim=1).argmax(dim=1)[0].float().cpu().numpy().astype(np.uint8)
    pred = cmap[output]
    
    visualize(batch_idx, input_image=img, ground_truth=gt, prediction=pred)
    
    if batch_idx == 10:
        break

### Save masked images and predictions

In [ ]:
# Load color map
cmap = np.load('datasets/NYUDV2/cmap.npy')

for batch_idx, data in enumerate(test_dataloader):
    img, target = data["partial_image1"].to(DEVICE), data["mask"].to(DEVICE) # partial_image1 instead of image
    output = torch.sigmoid(model(img.float()))
    
    img = (img[0].permute(1,2,0).detach().cpu().numpy()+1)/2
    img = (img*255).astype(np.uint8)
    img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

#     gt = target.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy()
#     gt=(gt*255).astype(np.uint8)
#     gt=cv2.cvtColor(gt,cv2.COLOR_RGB2BGR)
        
#     pred = output.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy() > 0.5
#     pred=(pred*255).astype(np.uint8)
#     pred=cv2.cvtColor(pred,cv2.COLOR_RGB2BGR)
    gt = target.squeeze().data.cpu().numpy()
    gt = cmap[gt]

    # pred = (
    #     cv2.resize(
    #         output[0, :40].data.cpu().numpy().transpose(1, 2, 0),
    #         target.size()[1:][::-1],
    #         interpolation=cv2.INTER_CUBIC,
    #     )
    #     .argmax(axis=2)
    #     .astype(np.uint8)
    # )

    output = torch.softmax(output, dim=1).argmax(dim=1)[0].float().cpu().numpy().astype(np.uint8)
    pred = cmap[output]
    
    cv2.imwrite(os.path.join(LOG_PATH, "vis_masked", "imgs/")+str(batch_idx)+'.png', img)
    cv2.imwrite(os.path.join(LOG_PATH, "vis_masked", "gts/")+str(batch_idx)+'.png', gt)
    cv2.imwrite(os.path.join(LOG_PATH, "vis_masked", "preds/")+str(batch_idx)+'.png', pred)
    
    
    #visualize(batch_idx, input_image=img, ground_truth=gt, prediction=pred)
    
    #if batch_idx == 20:
        #break